In [ ]:
from glob import glob
import shutil
import os

# for img in image_paths:
#     shutil.move(img, './a/')
# image_paths = glob("./a/*")

# image_paths = glob("C:\\Users\\mmumbaiwala\\Code\\TSR\\table-detection\\plots\\*")
image_paths = glob("C:\\Users\\mmumbaiwala\\Pictures\\det_new\\*")
len(image_paths)

In [ ]:
import IPython.display as ipd
from PIL import Image

from ultralyticsplus import YOLO, render_result
import ultralyticsplus



# # load model
# model = YOLO('foduucom/table-detection-and-extraction')

# # set model parameters
# model.overrides['conf'] = 0.25  # NMS confidence threshold
# model.overrides['iou'] = 0.45  # NMS IoU threshold
# model.overrides['agnostic_nms'] = False  # NMS class-agnostic
# model.overrides['max_det'] = 1000  # maximum number of detections per image

In [ ]:
import numpy as np
from typing import Optional, Union
import matplotlib.pyplot as plt
import cv2
import json
from tqdm import tqdm



# config = {
#     'conf' : 0.25,  # NMS confidence threshold
#     'iou' : 0.45,  # NMS IoU threshold
#     'agnostic_nms' : False,  # NMS class-agnostic
#     'max_det' : 1000,  # maximum number of detections per image
#     }

In [ ]:
class YoloTableDetector():
    def __init__(self, *, config:Optional[dict]=None):
        self.model = YOLO('foduucom/table-detection-and-extraction')

        if not config:
            config = {'conf' : 0.25,  # NMS confidence threshold
                    'iou' : 0.45,  # NMS IoU threshold
                    'max_det' : 1000,  # maximum number of detections per image
                    'agnostic_nms' : False,  # NMS class-agnostic
                }
                  
        for key, value in config.items():
            self.model.overrides[key] = value

            
    
    def detect(self, image:Union[str, np.ndarray]):
        # Verify image data format
        if isinstance(image, np.ndarray):
            if image.dtype!='uint8':
                raise TypeError(f"Image dtype {image.dtype} not supported, should be 'uint8'")

        
        detection_results = self.model.predict(image)
        return detection_results

    def save_plot_detections(
        self,
        image:Union[str, np.ndarray],
        *,
        save_path:str="",
        save_format:str='png',
        plot:bool=True,
        json_path:str=""
        ):
        
        detection_results = self.detect(image)[0] #Non streaming option


        

        if plot:
            if bool(detection_results.boxes):
                # for table in detection_results:
                render = render_result(model=self.model, image=image, result=detection_results)
                ipd.display(render)
            
        if save_path:
            # for table in detection_results:
                render = render_result(model=self.model, image=image, result=detection_results)
                
                render.save(save_path, forat=save_format)
                
        if json_path:
            json_data = process_results_json(detection_results, json_path)
            
            with open(json_path, 'w') as f:
                json.dump(json_data, f)
        return None


def process_results_json(detection_results, json_path:str=""):
    """results = self.detect(image)[0] #Non streaming option, see TableDetector init"""
    json_results = {}
    for i, (xyxy, conf) in enumerate(
                                zip(detection_results.boxes.xyxy,
                                    detection_results.boxes.conf
                                    )
                                ):
        json_results[i] = {'xyxy':[int(c) for c in xyxy], 'conf':float(conf.item())}

    return json_results
        

In [ ]:
td = YoloTableDetector()

In [ ]:
for i, img_path in enumerate(tqdm(image_paths)):
    # td.save_plot_detections(img_path, save_path=f"./{i}.png", json_path=f"./{i}.json")
    td.save_plot_detections(img_path, plot=True)

In [ ]:
# # set image
# # image = image_paths[5]
# # image = "C:\\Users\\mmumbaiwala\\Pictures\\det_new\\Screenshot 2023-08-28 164956.png"
# image = "C:\\Users\\mmumbaiwala\\Code\\TSR\\table-detection\\output_images\\page_2.png"
# td.save_plot_detections(image, plot=False, save_coordinates=True, json_path="./")
# # perform inference
# # image = plt.imread(image)[:,:,:3]
# # image = cv2.imread(image)[:,:,:3][:,:,::-1]
# results = model.predict(image)

# # td.save_plot_detections(image)
# # observe results
# print(results[0].boxes)
# render = render_result(model=model, image=image, result=results[0])
# # render.show()

# ipd.display(render)